### TC with Movie reviews

In [1]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [3]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews',
                                                  split=('train[60%:]', 'train[60%:]','test'),
                                                  as_supervised=True)

Shuffling and writing examples to C:\Users\fmeyer\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteGL7XYQ\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\fmeyer\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteGL7XYQ\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\fmeyer\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteGL7XYQ\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\fmeyer\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [5]:
# take it look into the training set
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'"54" is a film based on the infamous "Studio 54" of the 1970s - the hangout for the social elite and party clubbers. In the film, Ryan Phillippe is the main character, based on an actual employee of Studio from 1977 - 1982.<br /><br />The film\'s problem is that it\'s all glitter and style and no substance. It tries to be a really grimy and probing satire like "Boogie Nights" but ultimately comes across as an inferior wannabe. Mike Myers is given the thankless task of playing cocaine-snorting club owner Steve Rubell. It\'s only a slightly comedic role and if this was Myers\' best attempts at sliding into drama like Lemmon and other comedic actors did in their time, it\'s a total failure.<br /><br />"54" could have been insightful and interesting but instead it\'s just another dumb teen flick that isn\'t entertaining or even remotely engaging. View at your own peril.',
       b"Love Trap is not a short, it's quite obviously a full l

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1], dtype=int64)>

In [14]:
# use pretrained embeddings with tf hub to represent text

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype = tf.string, trainable = True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.0287814 , -3.09827   ,  3.0092616 , -1.6790327 , -1.9187448 ,
        -0.541796  , -1.3179585 , -0.2763995 ,  3.1945267 , -2.3923922 ,
        -2.158214  ,  0.36449632, -1.2040817 , -0.3907468 , -1.8369267 ,
         0.99000245,  1.6360253 , -0.8168775 , -1.80204   , -1.3412529 ],
       [ 3.3755968 , -3.909652  ,  5.5317564 , -0.34337172, -5.3195777 ,
        -3.975334  , -2.3213794 , -1.1221989 ,  4.1869936 , -2.7485137 ,
        -1.6916426 ,  0.37863284, -0.5454323 ,  0.746681  , -4.028013  ,
         0.15717079,  4.1928625 , -0.79877955, -2.7704723 , -2.4364202 ],
       [ 2.0856028 , -3.8065917 ,  3.6239567 , -1.6610895 , -3.7669764 ,
        -1.7204331 , -1.4448599 ,  1.3590558 ,  4.238418  , -0.31059918,
        -2.1275387 , -0.39205027, -0.4028368 ,  0.02771739, -3.4792125 ,
         0.557692  ,  3.860468  , -1.338282  , -2.3082013 , -0.8058568 ]],
      dtype=float32)>

In [15]:
#build a sequential model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
20/20 [==============================] - 2s 118ms/step - loss: 2.7906 - accuracy: 0.5023 - val_loss: 1.7743 - val_accuracy: 0.4983
Epoch 2/20
20/20 [==============================] - 2s 92ms/step - loss: 1.1202 - accuracy: 0.5145 - val_loss: 0.7127 - val_accuracy: 0.5747
Epoch 3/20
20/20 [==============================] - 2s 90ms/step - loss: 0.7049 - accuracy: 0.5635 - val_loss: 0.6804 - val_accuracy: 0.5732
Epoch 4/20
20/20 [==============================] - 2s 90ms/step - loss: 0.6552 - accuracy: 0.6093 - val_loss: 0.6358 - val_accuracy: 0.6320
Epoch 5/20
20/20 [==============================] - 2s 99ms/step - loss: 0.6202 - accuracy: 0.6429 - val_loss: 0.6026 - val_accuracy: 0.6530
Epoch 6/20
20/20 [==============================] - 2s 118ms/step - loss: 0.5904 - accuracy: 0.6675 - val_loss: 0.5770 - val_accuracy: 0.6817
Epoch 7/20
20/20 [==============================] - 2s 111ms/step - loss: 0.5684 - accuracy: 0.6884 - val_loss: 0.5576 - val_accuracy: 0.6989
Epoch 8/20

In [18]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.399
accuracy: 0.810


Extreme simple approach (small embedding dim, 2 FC layer to train --> still 0.81 Acc